In [10]:
# needed
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import mpld3
from modling.dataset import dataset
%matplotlib inline

mpld3.enable_notebook()
plt.style.use(['dark_background',])

# Loading the data files
* declare dataset object
* use object to load training set
* visualize head of training set 

In [11]:
ds = dataset()
ds.build_train_set()
ds.training_set.head()

FileNotFoundError: [WinError 3] The system cannot find the path specified: '..\\data_k'

## Goals
* Understand
* Run
* Model